# 🐟 Proyecto OrestiasGen – Pipeline Exploratoria Completa en Colab
Este notebook implementa el **pipeline completo exploratorio** descrito en el README, utilizando datos públicos.
Incluye:
- Instalación de herramientas
- Descarga de datos SRA (4 especies de *Orestias*)
- Control de calidad
- Alineamiento al genoma de referencia
- Llamado de SNPs
- Carga del VCF
- Estadística descriptiva
- PCA y visualizaciones

⚠️ *Este pipeline es exploratorio y opera con datos reales, por lo que puede tardar varios minutos en ejecutarse.*

## 🔧 Instalación de herramientas

In [ ]:
!apt-get update
!apt-get install -y sra-toolkit
!pip install scikit-allel pandas matplotlib seaborn biopython


## 🔐 Configuración de SRA

In [ ]:
!vdb-config --interactive


## 📁 Crear carpetas de trabajo

In [ ]:
import os

folders = ["data", "fastq", "qc", "bam", "vcf"]
for f in folders:
    os.makedirs(f, exist_ok=True)

folders

## 📥 Descarga de datos SRA
Se usarán accesiones ejemplo de *Orestias* (placeholder; puedes reemplazar por accesiones reales).

In [ ]:
sra_ids = [
    "SRR390728",  # EJEMPLO — reemplazar por Orestias
    "SRR390727",
    "SRR390726",
    "SRR390725"
]

for sra in sra_ids:
    !prefetch {sra}
    !fasterq-dump {sra} -O fastq/


## 🧼 Control de calidad con FastQC

In [ ]:
!fastqc fastq/*.fastq -o qc/

## 🧬 Descargar genoma de referencia (**Orestias ascotanensis**)

In [ ]:
!wget -O data/ore_ref.fasta https://ftp.ncbi.nlm.nih.gov/genomes/all/placeholder_reference.fasta


## 🧲 Indexar el genoma de referencia (BWA)

In [ ]:
!apt-get install -y bwa samtools bcftools
!bwa index data/ore_ref.fasta

## 🎯 Alineamiento de FASTQ a referencia

In [ ]:

import glob

fastqs = glob.glob("fastq/*.fastq")
for fq in fastqs:
    sample = fq.split("/")[-1].replace(".fastq","")
    !bwa mem data/ore_ref.fasta {fq} | samtools sort -o bam/{sample}.bam
    !samtools index bam/{sample}.bam


## 🧬 Llamado de SNPs con bcftools

In [ ]:

!samtools mpileup -uf data/ore_ref.fasta bam/*.bam | bcftools call -mv -Oz -o vcf/orestias_raw.vcf.gz
!bcftools index vcf/orestias_raw.vcf.gz


## 🔍 Filtrado básico de SNPs

In [ ]:

!bcftools filter -i 'QUAL>20 && DP>3' vcf/orestias_raw.vcf.gz -Oz -o vcf/orestias_filt.vcf.gz
!bcftools index vcf/orestias_filt.vcf.gz


## 🔬 Cargar VCF y explorar SNPs con scikit-allel

In [ ]:

import allel
import numpy as np

callset = allel.read_vcf("vcf/orestias_filt.vcf.gz")
gt = allel.GenotypeArray(callset['calldata/GT'])

print("N SNPs:", gt.shape[0])
print("N muestras:", gt.shape[1])


## 📈 Frecuencias alélicas

In [ ]:

ac = gt.count_alleles()
af = ac.to_frequencies()[:,1]
import matplotlib.pyplot as plt
plt.hist(af, bins=30)
plt.title("Distribución de frecuencias alélicas")
plt.show()


## 🧩 PCA – estructura genética

In [ ]:

gn = gt.to_n_alt()
gn_filled = np.nan_to_num(gn, nan=0)

coords, model = allel.pca(gn_filled, n_components=2)

plt.figure(figsize=(6,6))
plt.scatter(coords[:,0], coords[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA – Orestias (exploratorio)")
plt.show()
